In [13]:
import pandas as pd
data = {'text':[], 'label':[]}
df = pd.read_excel('./sampled.xlsx')
df['DATA'] = df['DATA'].fillna('')
for dt, label in zip(df['DATA'], df['TARGET']):
    try: 
        data['label'].append(int(label))
        data['text'].append(dt)
    except: pass
print(len(data['text']), len(data['label']))
df = pd.DataFrame(data)

499 499


In [14]:
df

,text,label
0,유럽에서는 겨울철만 되면 남여노소 뱅쇼를 즐긴다고 해요~ 와인에 과일을 넣고 푹끓...,1
1,빨간 순두부만 먹다가... 껍질벗긴 들깨가루를 넣어 영양을 더해보니 보양식이 따로 ...,1
2,고기와 양념만 있으면 어디서든 쉽게 만들어 먹을 수 있어요. 캠핑의 꽃은 고기!,0
3,아들이 먹고 싶다는 골뱅이무침에 소면을 곁들여 한끼 식사로 충분,0
4,사과는 식이섬유가 많고 다당류 비율이 높아서 다이어트에도 도움이 되며 특히 사과 ...,0
...,...,...
494,강판에 갈아서 만드는 감자전은 많이 알고 계실텐데요. 감자를 갈지 않고 채를 썰어서...,1
495,http://blog.naver.com/chez_sophie,0
496,파티에 튀김요리 없으면 허전하죠. 언제나 맛있는 튀김요리 모양도 이쁘고 완전 바삭...,0
497,남편이 좋아하는 시원한 재첩국을 미소 된장을 풀어 구수하고 담백하게 끓였는데 정말 ...,1


In [15]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset

# 데이터 준비 (예시)
# data = {
#     'text': [
#         '어묵김말이: 맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요',
#         '현미호두죽: 현미호두죽',
#         '참치삼각김밥: 맨날 편의점에서만 사먹는 삼각김밥~ 집에서 간편히 만들 수 있다는 사실~ 참치만 있으면 냉장고 열어서 만들 수 있어요~ 한끼식사로도 좋고 도시락으로도 좋은 참치삼각김밥을 만들어 보아요~'
#         # 추가 데이터 497개...
#     ],
#     'label': [1, 0, 0]  # 추가 데이터 라벨...
# }

# df = pd.DataFrame(data)

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# BERT 토크나이저 및 모델 로드
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=2)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# 데이터셋 및 데이터로더 준비
train_dataset = TextDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len=128)
test_dataset = TextDataset(X_test.tolist(), y_test.tolist(), tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 모델 훈련
trainer.train()

# 모델 평가
results = trainer.evaluate()
print("Evaluation results:", results)

# 예측 및 평가
preds_output = trainer.predict(test_dataset)
preds = preds_output.predictions.argmax(-1)

print("Accuracy:", accuracy_score(y_test, preds))
print("Classification Report:\n", classification_report(y_test, preds))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\user\miniconda3\envs\unet\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/500 [00:00<?, ?it/s]

{'loss': 0.6793, 'grad_norm': 2.848879098892212, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.4}
{'loss': 0.6818, 'grad_norm': 5.326639652252197, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.8}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.6732000112533569, 'eval_runtime': 1.2537, 'eval_samples_per_second': 79.765, 'eval_steps_per_second': 5.584, 'epoch': 1.0}
{'loss': 0.6833, 'grad_norm': 4.2678399085998535, 'learning_rate': 3e-06, 'epoch': 1.2}
{'loss': 0.6769, 'grad_norm': 4.341336250305176, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.6}
{'loss': 0.6717, 'grad_norm': 3.914797306060791, 'learning_rate': 5e-06, 'epoch': 2.0}


...

{'eval_loss': 1.6691598892211914, 'eval_runtime': 1.2943, 'eval_samples_per_second': 77.264, 'eval_steps_per_second': 5.408, 'epoch': 20.0}
{'train_runtime': 252.0105, 'train_samples_per_second': 31.665, 'train_steps_per_second': 1.984, 'train_loss': 0.43127016770839693, 'epoch': 20.0}


  0%|          | 0/7 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 1.6691598892211914, 'eval_runtime': 1.2965, 'eval_samples_per_second': 77.13, 'eval_steps_per_second': 5.399, 'epoch': 20.0}


  0%|          | 0/7 [00:00<?, ?it/s]

Accuracy: 0.66
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.81      0.73        58
           1       0.63      0.45      0.53        42

    accuracy                           0.66       100
   macro avg       0.65      0.63      0.63       100
weighted avg       0.66      0.66      0.65       100

